# Weather Agent

- Has a tool to fetch mock weather data

## Flow:

1. WeatherAgent generates weather data every cycle.
2. WeatherAgent sends a Task to AlertAgent via the Registry.
3. AlertAgent evaluates the data and triggers the alert tool if necessary.
4. Registry ensures both agents can discover each other.

In [ ]:
from protolink.agents import Agent
from protolink.models import AgentCard, Task
from protolink.transport import HTTPAgentTransport

URL = "http://localhost:8010"

# This could be an env variable
REGISTRY_URL = "http://localhost:9000"


class WeatherAgent(Agent):
    async def handle_task(self, task: Task):
        result = await self.call_tool("get_weather", city="Geneva")
        return task.complete(f"Weather data: {result}")


# -------------------------
# Transports
# -------------------------

# A2A Transport
transport = HTTPAgentTransport(url=URL)

# Define Agent Card using the Agent Card class
card = AgentCard(url=URL, name="WeatherAgent", description="Produces weather data")

agent = WeatherAgent(card=card, transport=transport, registry=REGISTRY_URL)

# -------------------------
# Tools
# -------------------------


# Add Native tool using the decorator
@agent.tool(name="get_weather", description="Return weather data for a city")
async def get_weather(city: str):
    # tool logic here
    return {"city": city, "temperature": 28, "condition": "sunny"}


await agent.start()